In [2]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from QuantumRingsLib import qasm2
from QuantumRingsLib import Parameter, ParameterVector

from random import random, uniform
from matplotlib import pyplot as plt
import numpy as np
import math
import time

import scipy
from scipy import optimize
import os
import math

provider = QuantumRingsProvider(token=os.environ.get('TOKEN_QUANTUMRINGS'), name=os.environ.get('ACCOUNT_QUANTUMRINGS'))
backend = provider.get_backend("scarlet_quantum_rings")
shots = 2

print(provider.active_account())

{'name': 'challenge4_team2@qai-ventures.com', 'token': 'rings-256.BT5Mmd8rd5fPx5K6p0Ff9bVyA9cUDON3', 'max_qubits': '256'}


In [3]:
# Implements a simple ansatz circuit using RY rotations
# Inputs: qc – QuantumCircuit
# theta_list – parameter list must be (reps + 1) * n_qubits wide.
# q – the qubit register to use
# n_qubits – Number of qubits to use
# reps – repeats of the circuit, defaults to 5
# insert_barriers –  whether to create a barrier gate
# Returns: Inplace modified QuantumCircuit

def SimpleAnsatz ( qc, q, n_qubits, theta_list, reps = 5, insert_barriers=False):
    for i in range (reps+1):
        theta = 0
        for j in range (n_qubits):
            #theta += theta_list[(i * n_qubits)+j]
            qc.ry(theta_list[(i * n_qubits)+j], q[j])

        if ( True == insert_barriers):
                qc.barrier()
    return

In [4]:
# Implements the ansatz circuit using RY rotations and CZ gates in a liner arrangement
# Inputs: qc – QuantumCircuit
# theta_list – rotational angle parameter must be (reps + 1) * n_qubits wide.
# q – the qubit register to use
# n_qubits – Number of qubits to use
# reps – repeats of the circuit, defaults to 5
# insert_barriers – whether to create a barrier gate after each rep.
# Returns: Inplace modified QuantumCircuit

def TwoLocalAnsatz ( qc, q, n_qubits, theta_list, reps=5, insert_barriers=False):
    i_theta = 0

    for i in range(n_qubits):
        qc.ry(theta_list[i_theta], q[i])
        i_theta += 1

    for _ in range(reps):
        if ( True == insert_barriers):
            qc.barrier()

        for i in range(n_qubits-1):
            qc.cz(q[i], q[i+1])

        for i in range(n_qubits):
            qc.ry(theta_list[i_theta], q[i])
            i_theta += 1
    return

In [5]:
# setup the Pauli operators corresponding to the problem Hamiltonian
# Inputs: nothing
# Returns: a list of Pauli operators and the corresponding weighing coefficients corresponding to the Hamiltonian


def get_Paulioperator():
    # Assume the following Hamiltonian
    #<H> = 2*<ψ(θ)|H1|ψ(θ)> + 4*<ψ(θ)|H2|ψ(θ)> + 8*<ψ(θ)|H3|ψ(θ)> + 16*<ψ(θ)|H4|ψ(θ)>
    #H1 = IIIZ
    #H2 = IIZZ
    #H3 = IZII
    #H4 = ZIIZ

    pauli_list = []
    w=[2+0j, 4+0j, 8+0j, 16+0j]

    pauli_list.append([w[0], "IIIZ"])
    pauli_list.append([w[1], "IIZZ"])
    pauli_list.append([w[2], "IZII"])
    pauli_list.append([w[3], "ZIIZ"])

    return pauli_list

In [6]:
# Performs pauli measurements
# qubitop: The Pauli operator
# param_dict: list of theta's for the parametrization circuit
# SHOTS: number of times, the circuit is to be repeated
def perform_pauli_measurements( qubitOp, param_dict, SHOTS=1024):

    avg = 0.0
    n_qubits = len(qubitOp[0][1])
    pauli_list = qubitOp


    # for each Pauli operator
    for p in pauli_list:
        weight = p[0].real
        pauli  = p[1]

        # assign parameters to the pqc and clone it
        qc = vqe_pqc.assign_parameters(param_dict)

        # Apply the Pauli operators.
        # no actions for "I" or "Z"
        for i in range(n_qubits):
            if (pauli[i] == "Y"):
                qc.sdg(q[i])
                qc.h(q[i])
            elif (pauli[i] == "X"):
                qc.h(q[i])

        # We should measure this circuit in the computation basis now
        qc.measure_all()

        job = backend.run(qc, shots= SHOTS, mode="sync", performance="HighestEfficiency", quiet=True)
        job_monitor(job, quiet=True)

        results = job.result()
        result_dict = results.get_counts()

        # perform the pauli measurement
        # convert the operator into binary
        measurement = 0.0
        pauli_int = int (p[1].replace("I","0").replace("Z","1").replace("X","1").replace("Y","1"),2)

        for key, value in result_dict.items():
            sign = -1.0 if ( bin(int(key,2) & pauli_int).count("1") & 1 ) else 1.0
            measurement += sign * value
        measurement /= SHOTS

        measurement = measurement * weight
        avg = avg + measurement

    return avg


# Given a theta_list, calculate the eigenstate
# Performs pauli measurements
# param_dict: list of theta's for the parametrization circuit
# SHOTS: number of times, the circuit is to be repeated
def find_eigenstate (param_dict, SHOTS=1024 ):

    # assign parameters to the pqc and clone it
    qc = vqe_pqc.assign_parameters(param_dict)

    # We should measure the circuit now
    qc.measure_all()

    job = backend.run(qc, shots= SHOTS, mode="sync", performance="HighestEfficiency", quiet=True)
    job_monitor(job, quiet=True)
    results = job.result()
    result_dict = results.get_counts()

    sorted_list = sorted(result_dict.items(), key=lambda value: value[1])

    return sorted_list[-1][0]

In [7]:
# Create the Parameterized Quantum Circuit
# Input:
# qubitop: The Pauli Operator
# theta_list: Parameter vector
# layers: number of circuit layers to use in the ansatz
# Returns: the Parameterized quantum circuit
# try both the ansatzes by uncommenting one of them

def create_ParameterizedVQECircuit(qubitOp, theta_list, layers):
    n_qubits = len(qubitOp[0][1])

    # construct the ansatz
    q = QuantumRegister(n_qubits, "q")
    c = ClassicalRegister(n_qubits, "c")
    circ = QuantumCircuit(q,c)

    #SimpleAnsatz (circ, q, n_qubits, theta_list, reps = layers)
    TwoLocalAnsatz (circ, q, n_qubits, theta_list, reps = layers)


    return circ

In [8]:
import json
from qiskit_aer import Aer
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.runtime import QAOA

# Step 1: Load company data from a JSON file
def load_company_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Step 2: Extract company names and ESG scores
def create_optimization_problem(companies):
    qp = QuadraticProgram()

    # Create binary variables for each company
    for company in companies:
        qp.binary_var(name=company['company'])  # Add binary decision variable

    # Set the objective to maximize the ESG scores
    qp.maximize(linear={company['company']: company['esg_score'] for company in companies})

    # Add a constraint (e.g., select at most one company)
    qp.linear_constraint({company['company']: 1 for company in companies}, '<=', 1)

    return qp

# Path to the JSON file
file_path = './outputs/company_esgscore.json'

# Load the company data
companies = load_company_data(file_path)

# Create the optimization problem
qp = create_optimization_problem(companies)

# Step 3: Convert the problem to QUBO form
qp_to_qubo = QuadraticProgramToQubo()
qubo = qp_to_qubo.convert(qp)

# Step 4: Define the backend
backend = Aer.get_backend('aer_simulator')

# Step 5: Define QAOA optimizer
qaoa = QAOA(backend=backend)

# Step 6: Create the Minimum Eigen Optimizer using QAOA
qaoa_optimizer = MinimumEigenOptimizer(qaoa)

# Step 7: Solve the optimization problem
result = qaoa_optimizer.solve(qubo)

# Step 8: Suggest the best company based on the result
selected_company = [companies[i]['company'] for i in range(len(companies)) if result.x[i] == 1]

# Step 9: Print the result (optimal portfolio)
print("Selected company based on ESG score:", selected_company)
print("Optimal ESG score:", result.fval)


ModuleNotFoundError: No module named 'qiskit_optimization.runtime'